In [1]:
!pip install pymongo


Defaulting to user installation because normal site-packages is not writeable


In [3]:
import json
import pymongo
from pymongo.errors import BulkWriteError
client=pymongo.MongoClient("mongodb://127.0.0.1:27017")


In [4]:
#Create database and collection 
db=client['students_data']
my_col=db['data']
with open("students.json","r")as file:##Loads the json file in database
    myObj1=json.load(file)

In [4]:
try:
  my_col.insert_many(myObj1)
except BulkWriteError as exc:
    var = exc.details

for i in my_col.find():
    print(i)

{'_id': 0, 'name': 'aimee Zank', 'scores': [{'score': 1.463179736705023, 'type': 'exam'}, {'score': 11.78273309957772, 'type': 'quiz'}, {'score': 35.8740349954354, 'type': 'homework'}]}
{'_id': 1, 'name': 'Aurelia Menendez', 'scores': [{'score': 60.06045071030959, 'type': 'exam'}, {'score': 52.79790691903873, 'type': 'quiz'}, {'score': 71.76133439165544, 'type': 'homework'}]}
{'_id': 2, 'name': 'Corliss Zuk', 'scores': [{'score': 67.03077096065002, 'type': 'exam'}, {'score': 6.301851677835235, 'type': 'quiz'}, {'score': 66.28344683278382, 'type': 'homework'}]}
{'_id': 3, 'name': 'Bao Ziglar', 'scores': [{'score': 71.64343899778332, 'type': 'exam'}, {'score': 24.80221293650313, 'type': 'quiz'}, {'score': 42.26147058804812, 'type': 'homework'}]}
{'_id': 4, 'name': 'Zachary Langlais', 'scores': [{'score': 78.68385091304332, 'type': 'exam'}, {'score': 90.2963101368042, 'type': 'quiz'}, {'score': 34.41620148042529, 'type': 'homework'}]}
{'_id': 5, 'name': 'Wilburn Spiess', 'scores': [{'scor

In [9]:
#student name who score maximum score in all the exam
stage2={"$unwind":{"path":"$scores"}}
stage1={"$group":{"_id":"$_id","name":{"$first":"$name"},"scores":{"$sum":"$scores.score"}}}
stage3={"$sort":{"scores":-1}}
stage4={"$limit":1}
stage5={"$project":{"_id":1,"name":1,"scores":1}}
for i in my_col.aggregate([stage2,stage1,stage3,stage4,stage5]):
    print(i)

{'_id': 13, 'name': 'Jessika Dagenais', 'scores': 275.94947751518237}


In [12]:
#Average mark for exam category
stage1={"$unwind":{"path":"$scores"}} 
stage2={"$match":{"scores.type":"exam"}}
stage3={"$group":{"_id":"$scores.type","avg":{"$avg":"$scores.score"}}}
for i in my_col.aggregate([stage1,stage2,stage3]):
    print(i)

{'_id': 'exam', 'avg': 48.67367075950175}


In [22]:
##Students who are below average in exam category
stage1={"$unwind":{"path":"$scores"}}
stage2={"$match":{"scores.type":"exam"}}
stage3={"$match":{"scores.score":{"$lt":49}}}
for i in my_col.aggregate([stage1,stage2,stage3]):
    print(i)

{'_id': 0, 'name': 'aimee Zank', 'scores': {'score': 1.463179736705023, 'type': 'exam'}}
{'_id': 5, 'name': 'Wilburn Spiess', 'scores': {'score': 44.87186330181261, 'type': 'exam'}}
{'_id': 6, 'name': 'Jenette Flanders', 'scores': {'score': 37.32285459166097, 'type': 'exam'}}
{'_id': 8, 'name': 'Daphne Zheng', 'scores': {'score': 22.13583712862635, 'type': 'exam'}}
{'_id': 10, 'name': 'Denisha Cast', 'scores': {'score': 45.61876862259409, 'type': 'exam'}}
{'_id': 14, 'name': 'Alix Sherrill', 'scores': {'score': 25.15924151998215, 'type': 'exam'}}
{'_id': 16, 'name': 'Dodie Staller', 'scores': {'score': 7.772386442858281, 'type': 'exam'}}
{'_id': 17, 'name': 'Fletcher Mcconnell', 'scores': {'score': 39.41011069729274, 'type': 'exam'}}
{'_id': 19, 'name': 'Gisela Levin', 'scores': {'score': 44.51211101958831, 'type': 'exam'}}
{'_id': 20, 'name': 'Tressa Schwing', 'scores': {'score': 42.17439799514388, 'type': 'exam'}}
{'_id': 21, 'name': 'Rosana Vales', 'scores': {'score': 46.22894762583

In [26]:
##students who score below pass mark in all category are assigned as pass
##students who score above pass mark in all category are assigned as fail
stage1={"$unwind":{"path":"$scores"}}
stage4={"$project":{"_id":1,"name":1,"scores.type":1,"grade":{"$cond":{"if":{"$gt":["$scores.score",40]},"then":"pass","else":"fail"}}}}
for i in my_col.aggregate([stage1,stage4]):
    print(i)

{'_id': 0, 'name': 'aimee Zank', 'scores': {'type': 'exam'}, 'grade': 'fail'}
{'_id': 0, 'name': 'aimee Zank', 'scores': {'type': 'quiz'}, 'grade': 'fail'}
{'_id': 0, 'name': 'aimee Zank', 'scores': {'type': 'homework'}, 'grade': 'fail'}
{'_id': 1, 'name': 'Aurelia Menendez', 'scores': {'type': 'exam'}, 'grade': 'pass'}
{'_id': 1, 'name': 'Aurelia Menendez', 'scores': {'type': 'quiz'}, 'grade': 'pass'}
{'_id': 1, 'name': 'Aurelia Menendez', 'scores': {'type': 'homework'}, 'grade': 'pass'}
{'_id': 2, 'name': 'Corliss Zuk', 'scores': {'type': 'exam'}, 'grade': 'pass'}
{'_id': 2, 'name': 'Corliss Zuk', 'scores': {'type': 'quiz'}, 'grade': 'fail'}
{'_id': 2, 'name': 'Corliss Zuk', 'scores': {'type': 'homework'}, 'grade': 'pass'}
{'_id': 3, 'name': 'Bao Ziglar', 'scores': {'type': 'exam'}, 'grade': 'pass'}
{'_id': 3, 'name': 'Bao Ziglar', 'scores': {'type': 'quiz'}, 'grade': 'fail'}
{'_id': 3, 'name': 'Bao Ziglar', 'scores': {'type': 'homework'}, 'grade': 'pass'}
{'_id': 4, 'name': 'Zachary

In [27]:
#Total and average of all category & store in seperate collection
stage1={"$unwind":{"path":"$scores"}}
stage2={"$group":{"_id":"$scores.type","Avg":{"$avg":"$scores.score"},"Total":{"$sum":"$scores.score"}}}
stage3={"$out":"Total_Avg"}

for i in my_col.aggregate([stage1,stage2,stage3]):
    print(i)

In [29]:
coll=db.list_collection_names()
print(coll)

['data', 'Total_Avg']


In [35]:
Total_Avg=db["Total_Avg"]

In [37]:
#Data stored in new collection

for i in Total_Avg.find():
    print(i)

{'_id': 'quiz', 'Avg': 48.99672319430254, 'Total': 9799.344638860508}
{'_id': 'exam', 'Avg': 48.67367075950175, 'Total': 9734.734151900351}
{'_id': 'homework', 'Avg': 67.81869620661149, 'Total': 13563.739241322297}


In [10]:
#Students who scored below average and above 40% in exam
stage1={"$unwind":{"path":"$scores"}}
stage2={"$match":{"scores.type":"exam"}}
stage3={"$match":{"$and":[{"scores.score":{"$gt":40}},{"scores.score":{"$lt":49}}]}}
stage4={"$group":{"_id":"$scores.score","name":{"$first":"$name"},"subject":{"$addToSet":"$scores.type"}}}
stage5={"$out":"B_Avg_A_Pass"}
for i in my_col.aggregate([stage1,stage2,stage3,stage4,stage5]):
    print(i)

In [11]:
#Students who scored below average and above 40% in quiz
stage1={"$unwind":{"path":"$scores"}}
stage2={"$match":{"scores.type":"quiz"}}
stage3={"$match":{"$and":[{"scores.score":{"$gt":40}},{"scores.score":{"$lt":49}}]}}
stage4={"$group":{"_id":"$scores.score","name":{"$first":"$name"},"subject":{"$addToSet":"$scores.type"}}}
stage5={"$merge":{"into":"B_Avg_A_Pass"}}
for i in my_col.aggregate([stage1,stage2,stage3,stage4,stage5]):
    print(i)

In [12]:
#Students who scored below average and above 40% in homework
stage1={"$unwind":{"path":"$scores"}}
stage2={"$match":{"scores.type":"homework"}}
stage3={"$match":{"$and":[{"scores.score":{"$gt":40}},{"scores.score":{"$lt":68}}]}}
stage4={"$group":{"_id":"$scores.score","name":{"$first":"$name"},"subject":{"$addToSet":"$scores.type"}}}
stage5={"$merge":{"into":"B_Avg_A_Pass"}}
for i in my_col.aggregate([stage1,stage2,stage3,stage4,stage5]):
    print(i)
    

In [13]:
#Students Data stored in new collection
col3=db["B_Avg_A_Pass"]
for i in col3.find():
    print(i)

{'_id': 40.15210496060384, 'name': 'Jenise Mcguffie', 'subject': ['exam']}
{'_id': 47.42608580155614, 'name': 'Demarcus Audette', 'subject': ['exam']}
{'_id': 40.68238966626067, 'name': 'Flora Duell', 'subject': ['exam']}
{'_id': 40.53136904234401, 'name': 'Quincy Danaher', 'subject': ['exam']}
{'_id': 44.29553481758053, 'name': 'Gwyneth Garling', 'subject': ['exam']}
{'_id': 42.89558347656537, 'name': 'Aleida Elsass', 'subject': ['exam']}
{'_id': 42.54322973844196, 'name': 'Laureen Salomone', 'subject': ['exam']}
{'_id': 44.87186330181261, 'name': 'Wilburn Spiess', 'subject': ['exam']}
{'_id': 41.3852820348269, 'name': 'Alica Pasley', 'subject': ['exam']}
{'_id': 42.17439799514388, 'name': 'Tressa Schwing', 'subject': ['exam']}
{'_id': 48.36644963899371, 'name': 'Gwyneth Garling', 'subject': ['exam']}
{'_id': 40.84572027366789, 'name': 'Chad Rahe', 'subject': ['exam']}
{'_id': 40.68676040665008, 'name': 'Malisa Jeanes', 'subject': ['exam']}
{'_id': 43.67436243299881, 'name': 'Alix She

In [18]:
#Students who scored below fail mark in exam
stage1={"$unwind":{"path":"$scores"}}
stage2={"$match":{"scores.type":"exam"}}
stage3={"$match":{"scores.score":{"$lt":40}}}
stage4={"$group":{"_id":"$scores.score","name":{"$first":"$name"},"subject":{"$addToSet":"$scores.type"}}}
stage5={"$out":"Below_fail"}
for i in my_col.aggregate([stage1,stage2,stage3,stage4,stage5]):
   print(i)

In [19]:
#Students who scored below fail mark in quiz
stage1={"$unwind":{"path":"$scores"}}
stage2={"$match":{"scores.type":"quiz"}}
stage3={"$match":{"scores.score":{"$lt":40}}}
stage4={"$group":{"_id":"$scores.score","name":{"$first":"$name"},"subject":{"$addToSet":"$scores.type"}}}
stage5={"$merge":{"into":"Below_fail"}}
for i in my_col.aggregate([stage1,stage2,stage3,stage4,stage5]):
   print(i)

In [20]:
#Students who scored below fail mark in homework
stage1={"$unwind":{"path":"$scores"}}
stage2={"$match":{"scores.type":"homework"}}
stage3={"$match":{"scores.score":{"$lt":40}}}
stage4={"$group":{"_id":"$scores.score","name":{"$first":"$name"},"subject":{"$addToSet":"$scores.type"}}}
stage5={"$merge":{"into":"Below_fail"}}
for i in my_col.aggregate([stage1,stage2,stage3,stage4,stage5]):
   print(i)

In [21]:
#Students Data stored in new collection
col4=db["Below_fail"]
for i in col4.find():
    print(i)

{'_id': 6.14281392478545, 'name': 'Sadie Jernigan', 'subject': ['exam']}
{'_id': 9.660849614328693, 'name': 'Dinah Sauve', 'subject': ['exam']}
{'_id': 1.463179736705023, 'name': 'aimee Zank', 'subject': ['exam']}
{'_id': 33.48242310776701, 'name': 'Santiago Dollins', 'subject': ['exam']}
{'_id': 19.96531774799065, 'name': 'Kim Xu', 'subject': ['exam']}
{'_id': 27.77924608896123, 'name': 'Echo Pippins', 'subject': ['exam']}
{'_id': 10.37320113489379, 'name': 'Grady Zemke', 'subject': ['exam']}
{'_id': 36.1767454709986, 'name': 'Rubie Winton', 'subject': ['exam']}
{'_id': 0.8007809823509016, 'name': 'Salena Olmos', 'subject': ['exam']}
{'_id': 32.99854612126559, 'name': 'Cody Strouth', 'subject': ['exam']}
{'_id': 39.19832917406515, 'name': 'Len Treiber', 'subject': ['exam']}
{'_id': 15.6237624645333, 'name': 'Jesusa Rickenbacker', 'subject': ['exam']}
{'_id': 8.125073097960179, 'name': 'Leonida Lafond', 'subject': ['exam']}
{'_id': 19.38544736721771, 'name': 'Alica Pasley', 'subject': 

In [23]:
#Students who scored Above Pass mark in exam
stage1={"$unwind":{"path":"$scores"}}
stage2={"$match":{"scores.type":"exam"}}
stage3={"$match":{"scores.score":{"$gt":40}}}
stage4={"$group":{"_id":"$scores.score","name":{"$first":"$name"},"subject":{"$addToSet":"$scores.type"}}}
stage5={"$out":"Above_Pass"}
for i in my_col.aggregate([stage1,stage2,stage3,stage4,stage5]):
   print(i)

In [24]:
#Students who scored Above Pass mark in quiz
stage1={"$unwind":{"path":"$scores"}}
stage2={"$match":{"scores.type":"quiz"}}
stage3={"$match":{"scores.score":{"$gt":40}}}
stage4={"$group":{"_id":"$scores.score","name":{"$first":"$name"},"subject":{"$addToSet":"$scores.type"}}}
stage5={"$merge":{"into":"Above_Pass"}}
for i in my_col.aggregate([stage1,stage2,stage3,stage4,stage5]):
   print(i)

In [25]:
#Students who scored Above Pass mark in homeWork
stage1={"$unwind":{"path":"$scores"}}
stage2={"$match":{"scores.type":"homework"}}
stage3={"$match":{"scores.score":{"$gt":40}}}
stage4={"$group":{"_id":"$scores.score","name":{"$first":"$name"},"subject":{"$addToSet":"$scores.type"}}}
stage5={"$merge":{"into":"Above_Pass"}}
for i in my_col.aggregate([stage1,stage2,stage3,stage4,stage5]):
   print(i)

In [26]:
col5=db["Above_Pass"]
for i in col5.find():
    print(i)

{'_id': 54.53947018434061, 'name': 'Tressa Schwing', 'subject': ['exam']}
{'_id': 90.47179954427436, 'name': 'Jessika Dagenais', 'subject': ['exam']}
{'_id': 69.54399888097534, 'name': 'Brittny Warwick', 'subject': ['exam']}
{'_id': 62.20457822364115, 'name': 'Zachary Langlais', 'subject': ['exam']}
{'_id': 51.38190070034149, 'name': 'Vina Matsunaga', 'subject': ['exam']}
{'_id': 95.45508256300009, 'name': 'Vinnie Auerbach', 'subject': ['exam']}
{'_id': 52.22632020277269, 'name': 'Tresa Sinha', 'subject': ['exam']}
{'_id': 64.47719204148157, 'name': 'Marcus Blohm', 'subject': ['exam']}
{'_id': 44.29553481758053, 'name': 'Gwyneth Garling', 'subject': ['exam']}
{'_id': 83.68438201130127, 'name': 'Jenise Mcguffie', 'subject': ['exam']}
{'_id': 69.60160495436016, 'name': 'Devorah Smartt', 'subject': ['exam']}
{'_id': 91.51968055194875, 'name': 'Dusti Lemmond', 'subject': ['exam']}
{'_id': 60.314725741828, 'name': 'Leola Lundin', 'subject': ['exam']}
{'_id': 60.06045071030959, 'name': 'Aure